### 필요한 모듈 imoprt

In [ ]:
# 필요한 모듈 import
import pandas as pd
import numpy as np
import re
import math

import matplotlib.pyplot as plt
import seaborn as sns

from scipy.spatial import cKDTree
from sklearn.neighbors import BallTree

import warnings
warnings.filterwarnings('ignore')

In [ ]:
plt.rcParams['font.family'] ='NanumGothic'
plt.rcParams['axes.unicode_minus'] =False

In [ ]:
# test 데이터 확인하기
test = pd.read_csv('./train-test/test_for_model_add_xy.csv', encoding='utf-8')

In [ ]:
test.info()

In [ ]:
# 학습 데이터 로드
df = pd.read_csv('./train-test/train_for_model_add_xy.csv', encoding='utf-8')

In [ ]:
df.info()

In [ ]:
# 계약년, 월 열 생성
df['계약년'] = df['계약년월'].astype('str').str[:4]
df['계약월'] = df['계약년월'].astype('str').str[4:]

In [ ]:
test['계약년'] = test['계약년월'].astype('str').str[:4]
test['계약월'] = test['계약년월'].astype('str').str[4:]

In [ ]:
df['계약년'].value_counts().plot(kind='bar') # 2015부터 사용
plt.title('연도별 데이터 분포')
plt.show()
# plt.savefig('./visualization/연도별 데이터 분포.png')

In [ ]:
df.groupby('계약년')['target'].mean().plot()
plt.title('연도별 매매가격 변화')
plt.show()
# plt.savefig('./visualization/연도별 데이터 분포.png')

### Feature Engineering

In [ ]:
def feature_engineering(df):
    # 내부 데이터로 파생변수 생성
    # 전용면적 로그 변환
    df["전용면적(log)"] = np.log1p(df["전용면적(㎡)"])

    # 전용면적 구간화
    df["전용면적구간"] = 0
    
    df.loc[df['전용면적(㎡)'] < 59, '전용면적구간'] = '59㎡미만'
    df.loc[(df["전용면적(㎡)"] >= 59) & (df["전용면적(㎡)"] < 60), '전용면적구간'] = '약59㎡'
    df.loc[(df["전용면적(㎡)"] >= 60) & (df["전용면적(㎡)"] < 84), '전용면적구간'] = '60㎡이상84㎡미만'
    df.loc[(df["전용면적(㎡)"] >= 84) & (df["전용면적(㎡)"] < 85), '전용면적구간'] = '약84㎡'
    df.loc[df['전용면적(㎡)'] >= 85, '전용면적구간'] = '85㎡이상'
    
    # 전용면적 평수 변환
    df["평수"] = df["전용면적(㎡)"] / 3.3

    # 연식(계약년-건축년도)
    df['계약년'] = df['계약년'].astype('int')
    df['연식'] = df['계약년'] - df['건축년도']
    df.loc[df['연식'] < 0, '연식'] = 0

    # 신축 / 재건축 라벨링
    df['신축(10년 미만)'] = (df['연식'] < 10).astype(int)
    df['재건축 연한(30년 이상)'] = (df['연식'] >= 30).astype(int)
    
    # 강남권 여부
    df['강남권여부'] = df['구'].apply(lambda x: 1 if x in ['강남구', '서초구', '송파구'] else 0)

    # 우수학군
    df['우수학군'] = df['구'].apply(lambda x: 1 if x in ['강남구', '양천구', '노원구', '서초구', '송파구', '마포구'] else 0)

    # 프리미엄아파트 여부
    df['프리미엄아파트'] = df['구'].apply(lambda x: 1 if x in ['강남구', '용산구', '성동구'] else 0)
    
    # # 층 구간화
    bins = [-15, 5, 15, float('inf')]
    labels = ["저층", "중층", "고층"]
    df["층"] = pd.cut(df["층"], bins = bins, labels = labels)

    # zone4 생성 및 원-핫 인코딩
    CBD = {'종로구','중구','용산구'}
    GANGNAM3 = {'강남구','서초구','송파구'}
    INNER = {'마포구','성동구','광진구','동대문구','서대문구','성북구','영등포구','양천구','동작구','관악구','강동구'}

    def map_zone(gu):
        if gu in CBD: return '도심'
        if gu in GANGNAM3: return '강남3'
        if gu in INNER: return '내부권'
        return '외곽'

    df['zone4'] = df['구'].map(map_zone)
    zone_dummies = pd.get_dummies(df['zone4'], prefix='zone4')
    df[zone_dummies.columns] = zone_dummies

    print(f'내부 데이터 feature engineering 완료: {df.shape}')
    return df

In [ ]:
print('train')
train = feature_engineering(df)
print('\ntest')
test = feature_engineering(test)

### 외부데이터 feature 추가

#### 1. 대장아파트

In [ ]:
# 대장아파트와 거리
def top_apt_dis(df):
    # 구별 대장아파트 목록
    lead_house = {
        "강서구" : (37.56520754904415, 126.82349451366355),
        "관악구" : (37.47800896704934, 126.94178722423047),
        "강남구" : (37.530594054209146, 127.0262701317293),
        "강동구" : (37.557175745977375, 127.16359581113558),
        "광진구" : (37.543083184171, 127.0998363490422),
        "구로구" : (37.51045944660659, 126.88687199829572),
        "금천구" : (37.459818907487936, 126.89741481874103),
        "노원구" : (37.63952738902813, 127.07234254197617),
        "도봉구" : (37.65775043994647, 127.04345013224447),
        "동대문구" : (37.57760781415707, 127.05375628992316),
        "동작구" : (37.509881249641495, 126.9618159122961),
        "마포구" : (37.54341664563958, 126.93601641235335),
        "서대문구" : (37.55808950436837, 126.9559315685538),
        "서초구" : (37.50625410912666, 126.99846468032919),
        "성동구" : (37.53870643389788, 127.04496220606433),
        "성북구" : (37.61158435092128, 127.02699796439015),
        "송파구" : (37.512817775046074, 127.08340371063358),
        "양천구" : (37.526754982736556, 126.86618704123521),
        "영등포구" : (37.52071403351804, 126.93668907644046),
        "용산구" : (37.521223570097305, 126.97345317787784),
        "은평구" : (37.60181702377437, 126.9362806808709),
        "종로구" : (37.56856915384472, 126.96687674967252),
        "중구" : (37.5544678205846, 126.9634879236162),
        "중랑구" : (37.58171824083332, 127.08183326205129),
        "강북구" : (37.61186335979484, 127.02822407466175)
    }
    
    # 위경도를 이용해 두 지점간의 거리를 구하는 함수를 생성
    def haversine_distance(lat1, lon1, lat2, lon2):
        radius = 6371.0
    
        lat1 = math.radians(lat1)
        lon1 = math.radians(lon1)
        lat2 = math.radians(lat2)
        lon2 = math.radians(lon2)
    
        dlon = lon2 - lon1
        dlat = lat2 - lat1
        a = math.sin(dlat / 2) ** 2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
        c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    
        distance = radius * c
        return distance

    # 대장 아파트의 위경도 데이터프레임을 구성
    lead_house_data = pd.DataFrame([{"구": k, "대장_좌표X": v[1], "대장_좌표Y": v[0]} for k, v in lead_house.items()])

    # 데이터프레임 간 결합
    df = df.merge(lead_house_data, how="left", on="구")

    # haversine_distance 함수를 이용해 대장아파트와의 거리를 계산, 새롭게 컬럼을 구성
    df['대장아파트거리'] = df.apply(lambda row: haversine_distance(row["y좌표"], row["x좌표"], row["대장_좌표Y"], row["대장_좌표X"]), axis=1)

    # 필요없는 열 빼기
    df = df.drop(['대장_좌표X', '대장_좌표Y'], axis=1)

    # 대장아파트거리 fe
    # 대장아파트거리 log scale
    df['대장아파트거리(log)'] = np.log1p(df['대장아파트거리'])

    # 대장아파트거리 구간화
    df.loc[df['대장아파트거리'] <= 0.3, '대장아파트거리접근성'] = 'A초근접'
    df.loc[(df['대장아파트거리'] > 0.3) & (df['대장아파트거리'] <= 0.8), '대장아파트거리접근성'] = 'B근접'
    df.loc[(df['대장아파트거리'] > 0.8) & (df['대장아파트거리'] <= 1.5), '대장아파트거리접근성'] = 'C중간'
    df.loc[(df['대장아파트거리'] > 1.5) & (df['대장아파트거리'] <= 3), '대장아파트거리접근성'] = 'D멀음'
    df.loc[(df['대장아파트거리'] > 3), '대장아파트거리접근성'] = 'E매우멀음'

    print(f'대장아파트 feature 생성: {df.shape}')
    return df

In [ ]:
print('train')
train = top_apt_dis(train)
print('\ntest')
test = top_apt_dis(test)

#### 2. 경제지표

#### 3. 교통

In [ ]:
# 버스, 지하철 거리(BallTree사용)
# bus, subway 데이터 불러오기
bus_df = pd.read_csv('./ml-outdata/bus_feature.csv', encoding='utf-8')
subway_df = pd.read_csv('./ml-outdata/subway_feature.csv', encoding='utf-8')

def bus_subway_feature(df, bus_df, subway_df):
    # df: ['y좌표', 'x좌표']
    # bus_df: ['Y좌표', 'X좌표']
    # subway_df: ['위도', '경도']

    # radian 변환 (위도 = y좌표, 경도 = x좌표)
    apt_coords = np.radians(df[['y좌표', 'x좌표']].values)
    bus_coords = np.radians(bus_df[['Y좌표', 'X좌표']].values)
    subway_coords = np.radians(subway_df[['위도', '경도']].values)
    
    # BallTree (하버사인 거리 사용)
    bus_tree = BallTree(bus_coords, metric='haversine')
    subway_tree = BallTree(subway_coords, metric='haversine')
    
    # 최근접 이웃 (k=1 → 가장 가까운 것 하나)
    dist_bus, idx_bus = bus_tree.query(apt_coords, k=1)
    dist_subway, idx_subway = subway_tree.query(apt_coords, k=1)
    
    # radian → km 변환
    R = 6371  # 지구 반지름 (km)
    df['버스거리'] = dist_bus[:, 0] * R
    df['지하철거리'] = dist_subway[:, 0] * R

    # 1km 이내 지하철 개수 feature
    # 1km -> radian 거리로 변환
    radius_km = 1.0
    radius_rad = radius_km / 6371  # 지구 반지름 R=6371km

    # query_radius 사용
    subway_count = subway_tree.query_radius(apt_coords, r=radius_rad, count_only=True)
    df['1km이내지하철수'] = subway_count

    print(f"1km 이내 지하철 개수 feature 추가 완료: min={subway_count.min()}, max={subway_count.max()}")

    # 지하철 거리 구간화 -> 접근성 feature 생성
    df['지하철접근성'] = np.select(
        [
            df['지하철거리'] <= 0.5,             # 5점: 500m 이내
            (df['지하철거리'] > 0.5) & (df['지하철거리'] <= 1.0),  # 4점: 0.5~1km
            (df['지하철거리'] > 1.0) & (df['지하철거리'] <= 2.0),  # 3점: 1~2km
            (df['지하철거리'] > 2.0) & (df['지하철거리'] <= 3.0)   # 2점: 2~3km
        ],
        [5, 4, 3, 2],
        default=1  # 3km 초과
    )
    
    # 분포 확인
    counts = df['지하철접근성'].value_counts().sort_index()
    ratios = counts / len(df) * 100
    
    print("\n 지하철접근성 5단계 분포 ")
    for 단계, cnt in counts.items():
        print(f"{단계}점: {cnt}개 ({ratios[단계]:.1f}%)")

    print(f'\n교통 feature 추가: {df.shape}')
    return df

In [ ]:
print('train')
train = bus_subway_feature(train, bus_df, subway_df)
print('\ntest')
test = bus_subway_feature(test, bus_df, subway_df)

#### 4. 학교

In [ ]:
# 학군 외부데이터 feature
# 학교 데이터 불러오기
schools = pd.read_csv('./ml-outdata/#서울시 학교 (초+중+고) - 시트1.csv', encoding='utf-8')
# 학교별 분리 (열 이름: '학교종류명', '위도', '경도')
el_schools = schools[schools['학교종류명'] == '초등학교']

# 진학률 데이터 불러오기 및 전처리
in_high_school = pd.read_csv('./ml-outdata/고등학교 진학률_특목고.csv', encoding='utf-8')
in_high_school = in_high_school.iloc[4:, :].rename(columns={'자치구별(2)':'구'})
in_high_school = in_high_school.reset_index(drop=True)
in_high_school = in_high_school.melt(id_vars=['구'], var_name='계약년', value_name='고등학교진학률')

def school_feature(df, schools, el_schools, in_high_school): 
    # 1km 이내 학교 수 feature
    # df['y좌표', 'x좌표']
    # schools['위도', '경도']
    # --------------------------
    
    # 위경도를 라디안으로 변환
    df_coords = np.radians(df[['y좌표', 'x좌표']].values)
    school_coords = np.radians(schools[['위도', '경도']].values)
    
    # BallTree 구축 (haversine metric)
    tree = BallTree(school_coords, metric='haversine')
    
    # 1km 반경 (반경 단위 = 라디안)
    radius_km = 1
    radius_rad = radius_km / 6371  # 지구 반지름 6371 km
    
    # 학교 수 계산
    school_counts = tree.query_radius(df_coords, r=radius_rad, count_only=True)
    df['1km이내학교수'] = school_counts
    
    print(f"1km 이내 학교 개수 feature 추가 완료: min={school_counts.min()}, max={school_counts.max()}")
    
    # 가장 가까운 초등학교 거리 feature
    # train: ['y좌표', 'x좌표']
    # el_schools: ['위도', '경도']
    
    # radian 변환 (위도 = y좌표, 경도 = x좌표)
    apt_coords = np.radians(df[['y좌표', 'x좌표']].values)
    school_coords = np.radians(el_schools[['위도', '경도']].values)
    
    # BallTree (하버사인 거리 사용)
    school_tree = BallTree(school_coords, metric='haversine')
    # 최근접 이웃 (k=1 → 가장 가까운 것 하나)
    dist_school, idx_school = school_tree.query(apt_coords, k=1)
    
    # radian → km 변환
    R = 6371  # 지구 반지름 (km)
    df['초등학교거리'] = dist_school[:, 0] * R

    # 초등학교를 도보로 갈 수 있는지 여부가 중요
    # 초등학교거리를 바탕으로 차등점수
    df['초등학교거리구분'] = 0
    
    df.loc[df['초등학교거리'] <= 0.3, '초등학교거리구분'] = 5
    df.loc[(df['초등학교거리'] > 0.3) & (df['초등학교거리'] <= 0.5), '초등학교거리구분'] = 4
    df.loc[(df['초등학교거리'] > 0.5) & (df['초등학교거리'] <= 0.7), '초등학교거리구분'] = 3
    df.loc[(df['초등학교거리'] > 0.7) & (df['초등학교거리'] <= 1), '초등학교거리구분'] = 2
    df.loc[df['초등학교거리'] > 1, '초등학교거리구분'] = 1

    # 분포 확인
    counts = df['초등학교거리구분'].value_counts().sort_index()
    ratios = counts / len(df) * 100
    
    print("\n 초등학교접근성 5단계 분포 ")
    for 단계, cnt in counts.items():
        print(f"{단계}점: {cnt}개 ({ratios[단계]:.1f}%)")
    
    # 진학률 데이터 merge
    # merge 위한 type 변경
    df['계약년'] = df['계약년'].astype('str')
    df = df.merge(in_high_school, how='left', on=['계약년', '구'])

    print(f'\n학교 feature 추가: {df.shape}')
    return df

EARTH_R = 6371  # km 단위로 계산

def add_elite_highschool_features(df, schools, elite_cats={'특목고','자율고'},
                                  lat_col='y좌표', lon_col='x좌표', radii=(1.5, 2.0)):
    """
    df: 아파트 데이터 (y좌표, x좌표)
    schools: 학교 데이터 (고등학교 구분명, 위도, 경도)
    elite_cats: 엘리트 고교 종류 집합
    radii: 반경 km 단위 리스트
    """
    # 1) 엘리트 고교 서브셋
    elite_df = schools[schools['고등학교 구분명'].isin(elite_cats)][['위도','경도']].dropna()
    
    if len(elite_df) == 0:
        # 엘리트 고교가 없으면 0 또는 NaN 채우기
        df['elite_min_dist_km'] = np.nan
        for r in radii:
            df[f'elite_cnt_{r}k'] = 0
        return df
    
    # 2) BallTree 구축
    pts_elite = np.c_[np.deg2rad(elite_df['위도'].values), np.deg2rad(elite_df['경도'].values)]
    tree = BallTree(pts_elite, metric='haversine')
    
    # 3) 아파트 좌표
    pts_apts = np.c_[np.deg2rad(df[lat_col].values), np.deg2rad(df[lon_col].values)]
    
    # 4) 최근접 거리 계산
    dist_rad, _ = tree.query(pts_apts, k=1)
    df['elite_min_dist_km'] = dist_rad[:,0] * EARTH_R  # rad -> km
    
    # 5) 반경 내 개수 계산
    for r in radii:
        r_rad = r / EARTH_R  # km -> rad
        counts = tree.query_radius(pts_apts, r=r_rad, count_only=True)
        df[f'elite_cnt_{r}k'] = counts.astype(int)
    
    print(f"엘리트 고교 feature 추가 완료: {df.shape}")
    print(f"최근접 거리(min={df['elite_min_dist_km'].min():.2f}km, max={df['elite_min_dist_km'].max():.2f}km)")
    for r in radii:
        print(f"{r}km 반경 내 개수: min={df[f'elite_cnt_{r}k'].min()}, max={df[f'elite_cnt_{r}k'].max()}")
    return df

In [ ]:
print('train')
train = school_feature(train, schools, el_schools, in_high_school)
train = add_elite_highschool_features(train, schools)
print('\ntest')
test = school_feature(test, schools, el_schools, in_high_school)
test  = add_elite_highschool_features(test,  schools)

#### 5. 시계열

In [ ]:
# 계약년월 가중치 열 추가
train['계약년월'] = train['계약년월'].astype('int')
test['계약년월'] = test['계약년월'].astype('int')

min_month = train['계약년월'].min()
train['계약년월가중치'] = train['계약년월'] - min_month
test['계약년월가중치'] = test['계약년월'] - min_month

In [ ]:
# 계약년, 계약월 int로 변경
train['계약년'] = train['계약년'].astype('int')
train['계약월'] = train['계약월'].astype('int')

test['계약년'] = test['계약년'].astype('int')
test['계약월'] = test['계약월'].astype('int')

#### 6. 집계함수

In [ ]:
def create_group_target_features_train_test(train_df, test_df, group_col, target_col, agg_funcs=None):
    """
    train 데이터로 그룹별 target 집계 feature를 만들고,
    test 데이터에도 같은 통계를 적용하는 함수
    """
    if agg_funcs is None:
        agg_funcs = ['mean', 'sum', 'max', 'min', 'count']
    
    for func in agg_funcs:
        new_col_name = f"{group_col}_{target_col}_{func}"
        
        # 1. train에서 집계 계산
        agg_map = train_df.groupby(group_col)[target_col].agg(func)
        
        # 2. train, test에 매핑
        train_df[new_col_name] = train_df[group_col].map(agg_map)
        test_df[new_col_name] = test_df[group_col].map(agg_map)
    
    return train_df, test_df

In [ ]:
train, test = create_group_target_features_train_test(train, test, group_col='구', target_col='target', agg_funcs=None)
train, test = create_group_target_features_train_test(train, test, group_col='전용면적구간', target_col='target', agg_funcs=None)
train, test = create_group_target_features_train_test(train, test, group_col='아파트명', target_col='target', agg_funcs=None)

In [ ]:
# target 데이터 뒤로 옮기기
cols = [c for c in train.columns if c != 'target'] + ['target']
train = train[cols]

In [ ]:
# 데이터 마지막 확인

In [ ]:
train.info()

In [ ]:
test.info()

### 모델 및 eda 데이터 생성

In [ ]:
# train.to_csv('train_2020.csv', encoding='utf-8', index=False)
# test.to_csv('test_2020.csv', encoding='utf-8', index=False)